In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from keras.src.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, multilabel_confusion_matrix, \
    mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from sklearn.neural_network import MLPRegressor

# Domains column Imputing
## Loading the dataset

In [ ]:
df = pd.read_excel('../../../data/BGG_Cleaned_Data_Set_Imputing.xlsx')

## Data preparation
#### Splitting the data

In [ ]:
targets = ['Strategy Games', 'Thematic Games', 'Wargames', 'Unknown', 'Family Games', 'Customizable Games',
           'Abstract Games', 'Party Games', 'Children\'s Games']

X = df.drop(targets, axis=1)
y = df[targets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Defining FNN model

In [ ]:
def create_bpnn():
    model = MLPRegressor(
        hidden_layer_sizes=(4096, 2056, 1028, 512, 256),
        activation='relu',
        solver='adam',
        max_iter=25,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=5,
        random_state=42
    )
    return model

#### Early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#### Learning rate scheduler

In [ ]:
def scheduler(epoch, lr):
    if epoch > 13:
        return lr * 0.5
    return lr

#### Callbacks for the lr adjustment

In [ ]:
callbacks = [LearningRateScheduler(scheduler), early_stopping]

#### Setting up the pipeline

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', create_bpnn())
])

#### Training the model

In [ ]:
pipeline.fit(X_train, y_train)

#### Make predictions

In [ ]:
y_pred = pipeline.predict(X_test)

## Model evaluation
#### Regression metrics

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

## Saving the model

In [ ]:
joblib.dump(pipeline, '../../../models/saved/domain_imputing_backward_propagation_model.pkl')